# Домашнее задание  № 5. Матричные разложения/Тематическое моделирование

In [2]:
import gensim
import pandas as pd
import numpy as np
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim_models
from collections import Counter
from string import punctuation
from razdel import tokenize as razdel_tokenize
from IPython.display import Image
from IPython.core.display import HTML 
from sklearn.decomposition import TruncatedSVD, NMF, PCA, LatentDirichletAllocation
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
from matplotlib import pyplot as plt
import seaborn as sns

morph = MorphAnalyzer()
warnings.filterwarnings("ignore")



/opt/anaconda3/lib/python3.7/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/opt/anaconda3/lib/python3.7/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/opt/anaconda3/lib/python3.7/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/opt/anaconda3/lib/python3.7/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, def

In [5]:
# добавим лемматизацию
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    normalized_text = [morph.parse(word)[0].normal_form for word in normalized_text]
    return ' '.join(normalized_text)


data = pd.read_csv('avito_category_classification.csv')
data['description_norm'] = data['description'].apply(normalize)
data = data[:5000]

### Задание № 1 (4 балла)

Попробуйте матричные разложения с 5 классификаторами - SGDClassifier, KNeighborsClassifier, MultinomialNB, RandomForest, ExtraTreesClassifier (про него подробнее почитайте в документации, он похож на RF). Используйте и NMF и SVD. Сравните результаты на кросс-валидации и выберите лучшее сочетание.

В итоге у вас должно получиться, как минимум 10 моделей (два разложения на каждый классификатор). Используйте 1 и те же параметры кросс-валидации. Параметры векторизации, параметры K в матричных разложениях, параметры классификаторов могут быть разными между экспериментами.

Можете взять поменьше данных, если все будет обучаться слишком долго (не ставьте параметр K слишком большим в NMF, иначе точно будет слишком долго)

In [17]:
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import TruncatedSVD, NMF

def eval_table(X, y, pipeline, N=6):
    labels = list(set(y))
    
    fold_metrics = pd.DataFrame(index=labels)
    errors = np.zeros((len(labels), len(labels)))
    kfold = StratifiedKFold(n_splits=N, shuffle=True, )
    
    for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
        pipeline.fit(X[train_index], y[train_index])
        preds = pipeline.predict(X[test_index])
        fold_metrics[f'f1_{i}'] = f1_score(y[test_index], preds, labels=labels, average=None)
        errors += confusion_matrix(y[test_index], preds, labels=labels)
    result = pd.DataFrame(index=labels)

    result['f1'] = fold_metrics[[f'f1_{i}' for i in range(N)]].mean(axis=1).round(2)

    return result['f1'].iloc[-1]

In [34]:
pipelines = []

method_dict = {
    TruncatedSVD(500): 'TruncatedSVD',
    NMF(100): 'NMF'
}

clf_dict = {
    SGDClassifier(): 'SGD', 
    KNeighborsClassifier(): 'KNN',
    RandomForestClassifier(): 'RF', 
    ExtraTreesClassifier(): 'ET'
#     MultinomialNB(): 'MNB'
}

for method in method_dict.keys():
    for clf in clf_dict.keys():
        ppln = Pipeline([
            ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
            ('svd', method),
            ('clf', clf)
        ])
        metrics = eval_table(data['description_norm'], data['category_name'], ppln)
        print(method_dict[method], '\t', clf_dict[clf], metrics)

TruncatedSVD SGD 0.96
TruncatedSVD KNN 0.75
TruncatedSVD RF 0.7
TruncatedSVD ET 0.46
NMF SGD 0.89
NMF KNN 0.8
NMF RF 0.89
NMF ET 0.91


Лучше справляется комбинация SVD + SGD, далее следуют NMF + SGD и NMF + RF

### Задание № 2 (6 баллов)

В Gensim тоже можно добавить нграммы и tfidf. Постройте 1 модель без них (как в семинаре) и еще 3 модели (1 с нграммами, 1 с tfidf и 1 с нграммами и с tfidf). Сранивте качество с помощью метрик (перплексия, когерентность) и на глаз. Определите лучшую модель. Для каждой модели выберите 1 самую красивую на ваш взгляд тему.

Используйте данные википедии из семинара. Можете взять поменьше данных, если все обучается долго.

Важное требование - получившиеся модели не должны быть совсем плохими. Если хороших тем не получается, попробуйте настроить гиперпараметры, отфильтровать словарь по-другому. 

Нграммы добавляются вот так (перед созданиеv словаря)

In [16]:
texts = []
with open('wiki_data.txt', 'r') as f:
    for line in f:
        texts.append(f.readline())

In [19]:
texts = [text.split() for text in texts]
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

# Без нграммов и TF-IDF

In [39]:
dictinary = gensim.corpora.Dictionary((text for text in texts))
dictinary.filter_extremes(no_above=0.1, no_below=10)
dictinary.compactify()

corpus = [dictinary.doc2bow(text) for text in texts]

print("Perplexity:", np.exp2(-lda.log_perplexity(corpus[:1000])))

lda_1 = gensim.models.LdaMulticore(corpus, 
                                 100, # колиество тем
                                 alpha='asymmetric',
                                 id2word=dictinary, 
                                 passes=10) 

topics = []
for topic_id, topic in lda_1.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=[text for text in texts], 
                                                   dictionary=dictinary, coherence='c_v')

print('Coherence:', coherence_model_lda.get_coherence())

Perplexity: 92522.5679418775
Coherence: 0.5632890737006185


In [40]:
lda_1.print_topics()[:10]

[(99,
  '0.010*"художник" + 0.010*"живописи" + 0.008*"века" + 0.007*"искусства" + 0.007*"в." + 0.007*"картины" + 0.006*"Армении" + 0.006*"художников" + 0.005*"художника" + 0.005*"либо"'),
 (98,
  '0.017*"СССР" + 0.005*"сборной" + 0.005*"чемпион" + 0.004*"Кубка" + 0.004*"бронзовым" + 0.004*"матери" + 0.004*"получил" + 0.004*"призёром" + 0.004*"мира" + 0.004*"составе"'),
 (97,
  '0.013*"ул." + 0.007*"здание" + 0.005*"строительства" + 0.005*"проекту" + 0.005*"части" + 0.004*"годов" + 0.004*"Н." + 0.004*"000" + 0.004*"трамвая" + 0.004*"П."'),
 (96,
  '0.086*"г." + 0.035*"гг." + 0.020*"Совета" + 0.016*"ЦК" + 0.014*"комитета" + 0.013*"председатель" + 0.013*"заместитель" + 0.012*"секретарь" + 0.012*"председателя" + 0.011*"государственный"'),
 (94,
  '0.062*"поселения" + 0.034*"территории" + 0.026*"сельское" + 0.024*"поселение" + 0.015*"образование" + 0.013*"муниципальное" + 0.011*"района" + 0.011*"Тверской" + 0.011*"центр" + 0.010*"сельского"'),
 (95,
  '0.004*"полк" + 0.004*"июля" + 0.004*"в

# С нграммами

In [59]:
len(corpus[1])

4

In [60]:
dictinary = gensim.corpora.Dictionary((text for text in ngrammed_texts))
dictinary.filter_extremes(no_above=0.05, no_below=10)
dictinary.compactify()

corpus = [dictinary.doc2bow(text) for text in ngrammed_texts]

print("Perplexity:", np.exp2(-lda.log_perplexity(corpus[:1000])))


lda_2 = gensim.models.LdaMulticore(corpus, 
                                 100, # колиество тем
                                 alpha='asymmetric',
                                 id2word=dictinary, 
                                 passes=10) 

topics = []
for topic_id, topic in lda_2.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=[text for text in ngrammed_texts], 
                                                   dictionary=dictinary, coherence='c_v')

print('Coherence:', coherence_model_lda.get_coherence())

Perplexity: 1725370.4906753823
Coherence: 0.548494989212285


In [62]:
lda_2.print_topics()[:10]

[(99,
  '0.015*"деревня" + 0.012*"км_к" + 0.010*"дивизия" + 0.009*"был_преобразован" + 0.008*"дивизии" + 0.008*"муниципальное_образование" + 0.007*"1_января" + 0.007*"1929_году" + 0.007*"Алнашском_районе" + 0.007*"наделён_статусом"'),
 (97,
  '0.023*"подвески" + 0.019*"автомобиля" + 0.009*"автомобиль" + 0.008*"автомобилей" + 0.005*"версия" + 0.005*"оси" + 0.005*"задней" + 0.005*"колеса" + 0.005*"сравнительно" + 0.005*"колесо"'),
 (98,
  '0.008*"колхоз" + 0.008*"колхоза" + 0.006*"центральная_усадьба" + 0.005*"ноября_2004" + 0.005*"1859" + 0.004*"(колхоз)" + 0.004*"переименован" + 0.004*"проживали" + 0.004*"жителей_мужского" + 0.003*"вошёл"'),
 (96,
  '0.008*"группы" + 0.008*"Сергей" + 0.008*"песни" + 0.006*"Александр" + 0.005*"Владимир" + 0.005*"Ирина" + 0.004*"Игорь" + 0.004*"альбом" + 0.004*"ансамбль" + 0.004*"группа"'),
 (94,
  '0.010*"Гитлер" + 0.006*"Гитлера" + 0.005*"команды." + 0.005*"команды" + 0.004*"1998" + 0.004*"был_признан" + 0.004*"де" + 0.003*"()" + 0.003*"иезуитов" + 0.0

# С TF–IDF

In [66]:
dictinary = gensim.corpora.Dictionary((text for text in texts))
dictinary.filter_extremes(no_above=0.01, no_below=10)
dictinary.compactify()

corpus = [dictinary.doc2bow(text) for text in texts]

tfidf = gensim.models.TfidfModel(corpus, id2word=dictinary, smartirs='lfc')
corpus = tfidf[corpus]

print("Perplexity:", np.exp2(-lda.log_perplexity(corpus[:1000])))

lda_3 = gensim.models.LdaMulticore(corpus, 
                                 100, # колиество тем
                                 alpha='asymmetric',
                                 id2word=dictinary, 
                                 passes=10) 

topics = []
for topic_id, topic in lda_3.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=[text for text in texts], 
                                                   dictionary=dictinary, coherence='c_v')

print('Coherence:', coherence_model_lda.get_coherence())

Perplexity: 1.2862625919507363e+61
Coherence: 0.4114695081463832


In [67]:
lda_3.print_topics()[:10]

[(99,
  '0.000*"Billboard" + 0.000*"поднялся" + 0.000*"барабанщика" + 0.000*"альбомов," + 0.000*"соло" + 0.000*"одноименной" + 0.000*"композиция" + 0.000*"Майкл" + 0.000*"коллектива" + 0.000*"годами"'),
 (98,
  '0.009*"IBM" + 0.000*"F" + 0.000*"во-первых," + 0.000*"печатный" + 0.000*"признаётся" + 0.000*"латинских" + 0.000*"T" + 0.000*"возродить" + 0.000*"U" + 0.000*"даны"'),
 (97,
  '0.000*"судно" + 0.000*"судном" + 0.000*"верфи" + 0.000*"пассажирский" + 0.000*"продана" + 0.000*"корабль" + 0.000*"корабля" + 0.000*"покинула" + 0.000*"пассажирских" + 0.000*"вокзала"'),
 (96,
  '0.000*"записал" + 0.000*"совместный" + 0.000*"альбомов" + 0.000*"итальянской" + 0.000*"del" + 0.000*"La" + 0.000*"написаны" + 0.000*"песню" + 0.000*"музыки" + 0.000*"(1994),"'),
 (95,
  '0.007*"мост" + 0.000*"мосту" + 0.000*"моста" + 0.000*"ограждение" + 0.000*"сечения" + 0.000*"юга" + 0.000*"моста." + 0.000*"завершается" + 0.000*"архитектурные" + 0.000*"речки"'),
 (93,
  '0.010*"серебряных" + 0.000*"Речи" + 0.00

# С нграммами и TF–IDF

In [79]:
dictinary = gensim.corpora.Dictionary((text for text in ngrammed_texts))
dictinary.filter_extremes(no_above=0.05, no_below=10)
dictinary.compactify()

corpus = [dictinary.doc2bow(text) for text in ngrammed_texts]

tfidf = gensim.models.TfidfModel(corpus, id2word=dictinary, smartirs='lfc')
corpus = tfidf[corpus]

print("Perplexity:", np.exp2(-lda.log_perplexity(corpus[:1000])))

lda_4 = gensim.models.LdaMulticore(corpus, 
                                 50, # колиество тем
                                 alpha='asymmetric',
                                 id2word=dictinary, 
                                 passes=10) 

topics = []
for topic_id, topic in lda_4.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=[text for text in ngrammed_texts], 
                                                   dictionary=dictinary, coherence='c_v')

print('Coherence:', coherence_model_lda.get_coherence())

Perplexity: 1.959383564087525e+50
Coherence: 0.4139797118671423


In [80]:
lda_4.print_topics()[:]

[(49,
  '0.000*"открытым" + 0.000*"были_сняты" + 0.000*"был_снят" + 0.000*"1889" + 0.000*"Снят" + 0.000*"длился" + 0.000*"Джей" + 0.000*"села" + 0.000*"игра" + 0.000*"ранее."'),
 (48,
  '0.000*"Марии" + 0.000*"заметных" + 0.000*"сотрудничать" + 0.000*"Кристина" + 0.000*"факультет," + 0.000*"начала_работать" + 0.000*"государственный_университет" + 0.000*"простое" + 0.000*"игровом" + 0.000*"документального"'),
 (47,
  '0.000*"вождя" + 0.000*"Узнав_об" + 0.000*"продвинулся" + 0.000*"добычей" + 0.000*"столицу," + 0.000*"отступили" + 0.000*"вторжение" + 0.000*"вторглись" + 0.000*"период," + 0.000*"закончилось"'),
 (46,
  '0.000*"был," + 0.000*"ты" + 0.000*"наладить" + 0.000*"Советский" + 0.000*"контакты" + 0.000*"аэропорт" + 0.000*"разгрома" + 0.000*"разведки" + 0.000*"входящих" + 0.000*"Великой_Отечественной"'),
 (45,
  '0.000*"прочие" + 0.000*"стиле" + 0.000*"комедия" + 0.000*"сооружений," + 0.000*"всякий" + 0.000*"решается" + 0.000*"охотников" + 0.000*"лишним" + 0.000*"творческий" + 0.00

Tfidf добавляется вот так (после векторизации и перед обучением lda)

# Результаты

## Темы
Все модели периодически выдают интересные темы, вот список тем, которые показались мне интересными:

'0.010*"художник" + 0.010*"живописи" + 0.008*"века" + 0.007*"искусства" + 0.007*"в." + 0.007*"картины" + 0.006*"Армении" + 0.006*"художников" + 0.005*"художника" + 0.005*"либо"'),

'0.004*"полк" + 0.004*"июля" + 0.004*"войны" + 0.003*"через" + 0.003*"1919" + 0.003*"ноября" + 0.003*"получил" + 0.003*"октября" + 0.003*"августа" + 0.003*"15"'),

'0.014*"марки" + 0.014*"ди" + 0.010*"марок" + 0.009*"Португалии" + 0.007*"почтовых" + 0.006*"Дании" + 0.006*"1974" + 0.006*"почтовой" + 0.005*"1975" + 0.005*"Canada"')]

  '0.023*"подвески" + 0.019*"автомобиля" + 0.009*"автомобиль" + 0.008*"автомобилей" + 0.005*"версия" + 0.005*"оси" + 0.005*"задней" + 0.005*"колеса" + 0.005*"сравнительно" + 0.005*"колесо"'),

  '0.000*"Billboard" + 0.000*"поднялся" + 0.000*"барабанщика" + 0.000*"альбомов," + 0.000*"соло" + 0.000*"одноименной" + 0.000*"композиция" + 0.000*"Майкл" + 0.000*"коллектива" + 0.000*"годами"'),

## Перплексия
Без всего: 92522.5679418775

Нграммы: 1725370.4906753823

TF–IDF: 1.2862625919507363e+61

Нграммы + TF–IDF: 1.959383564087525e+50

## Когерентность
Без всего: 0.5632890737006185

Нграммы: 0.548494989212285

TF–IDF: 0.4114695081463832 

Нграммы + TF–IDF: 0.4139797118671423

В моделях с использованием нграммов / TF–IDF снижается качество метрик, но некоторые темы становятся более точными